In [15]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [16]:
DATA_PATH = '../data'
DATA_FILE_INTERVIEWS = os.path.join(DATA_PATH, 'coded_interviews.csv')

In [17]:
data = pd.read_csv(DATA_FILE_INTERVIEWS) 

In [18]:
data

,Participant_ID,Age,Domicile,Marital_Status,Children_Count,Occupation,Health_Insurance,Employment_Status,Family_Doctor,Primary_Healthcare_Use,Private_Healthcare_Use,Education_Level,Heard_of_Cervical_Cancer,Heard_of_Testing_Programs,Performed_Preventive_Tests,Pap_Smear_Frequency,Last_Pap_Smear,Heard_of_HPV_Test,Performed_HPV_Test,Pre_Test_Information_Source,Project_Information_Source,Reaction_to_Positive_Result,Consulted_with_Others,Follow_Up_Actions,Interpretation_Aid,Current_Recommendations,Sample_Collection_Method,Overall_Experience,Barriers_to_Testing,Barriers_to_Follow_Up,Perception_of_Healthcare_Services,Preventive_Healthcare_Attitude,Knowledge_Acquisition_Method,Perceived_Barriers_in_Healthcare,Impact_of_Urban_vs_Rural,Support_System,Medical_Conditions,Economic_Barriers,Transportation_Issues,Responsibility,Healthcare_Seeking_Behavior,Overall_Insight
0,2,43,Rural,Single,7,Housewife,Yes,Social Assistance,Yes,Yes,No,Seventh Grade,Yes,No,Yes,Annual,2023,Yes,No,Midwife,Doctor,Disbelief and sadness,Yes,Ongoing,No,Hospitalization,Pap smear,Overwhelming at first,Economic,Lack of specialists,Positive,Preventive approach,Internet,Financial,Rural disadvantaged,Family,Asthma,Yes,Moderate,High,Proactive,Revealing and educational
1,3,43,Urban,Married,2,Expert in European funds,Yes,Employed,Yes,Yes,Yes,College,Yes,Yes,Yes,Sporadic,November 2023,Yes,Yes,Internet,Word of mouth,Shock and panic,Yes,Visited Cluj,No,Colposcopy,Pap smear,Overwhelming at first,NaN,Not significant,Positive,Preventive approach,Online research,Ignorance,Urban advantage,Family,NaN,NaN,NaN,High,Proactive,Revealing
